In [1]:
import torch
import gc
import numpy as np
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer, util

2025-04-28 23:11:54.859361: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745881915.164887      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745881915.242360      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
from huggingface_hub import login

### Import text for LLM

In [5]:
import requests

url = 'https://raw.githubusercontent.com/kvufee/itmo_ml_kaggle/refs/heads/main/hw7/data.txt'

data = requests.get(url).text

In [6]:
data = data.replace('\n', ' ').replace('  ', ' ')
data

'Как устроено самое мощное семейство не-нейросетевых моделей: градиентный бустинг над решающими деревьями В прошлых разделах мы научились соединять базовые алгоритмы в ансамбль с помощью бэггинга (и, в частности, строить из решающих деревьев случайные леса). Теперь мы рассмотрим другой способ объединять базовые алгоритмы в композицию — градиентный бустинг. В ходе обучения случайного леса каждый базовый алгоритм строится независимо от остальных. Бустинг, в свою очередь, воплощает идею последовательного построения линейной комбинации алгоритмов. Каждый следующий алгоритм старается уменьшить ошибку текущего ансамбля. Бустинг, использующий деревья решений в качестве базовых алгоритмов, называется градиентным бустингом над решающими деревьями, (Gradient Boosting on Decision Trees, GBDT). Он отлично работает на выборках с «табличными», неоднородными данными. Пример таких данных — описание пользователя Яндекса через его возраст, пол, среднее число поисковых запросов в день, число заказов такс

### LLM bot

In [21]:
def load_model(model_name):
    print(f"Loading {model_name}...")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

In [28]:
opt_facebook_model = "facebook/opt-1.3b"
falcon_model = "euclaise/falcon_1b_stage2"

In [29]:
model1, tokenizer1 = load_model(opt_facebook_model)
model2, tokenizer2 = load_model(falcon_model)

Loading facebook/opt-1.3b...
Loading euclaise/falcon_1b_stage2...


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

In [43]:
current_model = model1
current_tokenizer = tokenizer1

Make embeddings

In [39]:
context_text = f"""{data}"""
embedder = SentenceTransformer('all-MiniLM-L6-v2')
context_embedding = embedder.encode(data, convert_to_tensor=True)

def get_context(question):
    question_embedding = embedder.encode(question, convert_to_tensor=True)
    score = util.pytorch_cos_sim(question_embedding, context_embedding)
    return context_text if score > 0.5 else ""

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

### Create bots

In [50]:
def OPTBot():
    print("\nType 'exit' to quit")
    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() == "exit":
            break

        try:
            temperature = float(input("Temperature (default 0.7): ") or "0.7")
            max_new_tokens = int(input("Max new tokens (default 150): ") or "150")
        except:
            print("Invalid input, using defaults.")
            temperature = 0.7
            max_new_tokens = 150

        relevant_context = get_context(user_input)
        full_prompt = f"{relevant_context}\n\nQuestion: {user_input}\nAnswer:"

        inputs = tokenizer1(full_prompt, return_tensors="pt").to(model1.device)
        output = model1.generate(
            **inputs, 
            temperature=temperature,
            max_new_tokens=max_new_tokens,
            top_p=0.95,
            do_sample=True,
        )
        decoded = tokenizer1.decode(output[0], skip_special_tokens=True)
        answer = decoded.split("Answer:")[-1].strip()
        print(f"\nBot: {answer}")

In [51]:
def FalconBot():
    print("\nType 'exit' to quit")
    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() == "exit":
            break
        
        try:
            temperature = float(input("Temperature (default 0.7): ") or "0.7")
            max_new_tokens = int(input("Max new tokens (default 150): ") or "150")
        except:
            print("Invalid input, using defaults.")
            temperature = 0.7
            max_new_tokens = 150

        relevant_context = get_context(user_input)
        full_prompt = f"{relevant_context}\n\nQuestion: {user_input}\nAnswer:"

        inputs = tokenizer2(full_prompt, return_tensors="pt").to(model2.device)
        output = model2.generate(
            **inputs, 
            temperature=temperature,
            max_new_tokens=max_new_tokens,
            top_p=0.95,
            do_sample=True,
        )
        decoded = tokenizer2.decode(output[0], skip_special_tokens=True)
        answer = decoded.split("Answer:")[-1].strip()
        print(f"\nBot: {answer}")

In [52]:
FalconBot()


Type 'exit' to quit



You:  пример неоднородных данных
Temperature (default 0.7):  
Max new tokens (default 150):  300


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Bot: Раз постоянно больше представлены данных дачи в этом море игры которые возвращают не становят данных дачных с примером неоднородных данных, возвращают в море игры которые показывают при постепении данных данных с примером неоднородных данных с удобным игрым данных, возвращ


...


You:  что мы научились реализовывать в прошлых уроках
Temperature (default 0.7):  0.7
Max new tokens (default 150):  150


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Bot: что мы научились в прошлых уроках
Question: что я прошла в предложение при этом подоговарял требуется переведёние при оцене в причем при исполь



You:  exit


In [53]:
OPTBot()


Type 'exit' to quit



You:  пример неоднородных данных
Temperature (default 0.7):  
Max new tokens (default 150):  300


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Bot: Неоднородные данные выявили решение, что в конце концов примерно трех городов на себя было возможность предотвратить возможность пользователям на подконтрольную политику.

Question: в конце концов примерно трех городов на себя было возможность предотвратить возможность пользователям на подкон



You:  что такое бустинг
Temperature (default 0.7):  0.1
Max new tokens (default 150):  150


...


Bot: что такое бустинг

Спасибо за пределами представителей представителей представителей представителей представителей представителей представителей представителей представителей представителей представителей представителей представител



You:  что мы научились реализовывать в прошлых уроках
Temperature (default 0.7):  0.7
Max new tokens (default 150):  150


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Bot: Прошлый показатель, чтобы предотвратить не получать ничего не особенности, а особенности не было случайно определенной в этом случае, а о



You:  exit
